In [1]:
import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": go.Layout(title="hello world")
})

In [3]:
import igraph as ig
import numpy as np

In [15]:
src = np.random.randint(10, size=10)
dst = np.random.randint(10, size=10)
edges = list(zip(src, dst))
edges

[(8, 2),
 (4, 7),
 (8, 8),
 (5, 9),
 (3, 3),
 (7, 7),
 (4, 1),
 (3, 4),
 (4, 8),
 (8, 9)]

In [17]:
G=ig.Graph(edges, directed=False)
layt=G.layout('kk', dim=3) 

In [27]:
N = 10
Xn=layt[:,0]
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]  
    Ze+=[layt[e[0]][2],layt[e[1]][2], None] 

TypeError: list indices must be integers or slices, not tuple

In [28]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             #color=group,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               #text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of coappearances of characters in Victor Hugo's novel<br> Les Miserables (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
        )

data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='Les-Miserables')

In [172]:
from collections import UserDict

class UniqueDict(UserDict):
        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def add(self, x):
        try:
            return self.data[x]
        except KeyError:
            n = len(self.data)
            self.data[x] = n
            return n
        
    def to_list(self):
        return sorted(self, key=self.get)
    

class RecNet():
    
    def __init__(self, seq):
        edges = []
        nodes = []
        tokens = []
        token_edges = []
        unique_tokens = UniqueDict()
        for t, tok in enumerate(map(frozenset, seq)):
            if not tok:
                continue
                
            tok_id = unique_tokens.add(tok)
            
            matched = set()
            for d, tok_prev in enumerate(reversed(tokens)):
                common = tok & tok_prev
                if not common:
                    continue
                if common & matched:
                    continue
                
                edges.append((t-d-1, t, common))
                matched.update(common)
                
                if common == tok:
                    break
            
            tokens.append(tok)
            nodes.append(tok_id)
        
        
        self.nodes = np.array(nodes)
        self.tokens = tokens
        self.unique_tokens = np.array(unique_tokens.to_list())
        self.weights = np.array([w for _,_,w in edges])
        self.edges = np.array([(t0,t1) for t0,t1,_ in edges])
        
        def t_to_id(t):
            #= lambda t: 
            print("t=",t,end=" ; ")
            print("self.tokens[t]=",self.tokens[t])
            return unique_tokens[self.tokens[t]]
        self.edges_token_ids = np.array([(t_to_id(t0), t_to_id(t1)) for t0, t1 in self.edges])
        
    def summary(self, verbose_level=2):
        fs_fmt = lambda fs: "{" + ','.join(fs) + "}"
        list_fmt = lambda xs: "[" + ', '.join(xs) + "]"
        if verbose_level > 1:
            print("cascade identifiers:", list_fmt(fs_fmt(t) for t in self.unique_tokens))
        print(len(self.unique_tokens), "cascade identifiers")
        print(len(self.nodes), "nodes")
        if verbose_level > 1:
            print("edges:", list_fmt(f"{s}-{d} {fs_fmt(w)}" for (s, d), w in zip(self.edges, self.weights)))
        print(len(self.edges), "edges")
        
    def plot(self):
        markers = self.nodes
        
        import plotly.plotly as py
        import plotly.graph_objs as go

        Ytok = np.random.random([len(self.unique_tokens)])
        Yn = Ytok[markers]
        
        edge_trace = go.Scatter(
            x=[],
            y=[],
            line=dict(width=0.5,color='#888'),
            hoverinfo='none',
            mode='lines')

        for (t0,t1), (n0, n1), w in zip(self.edges, self.edges_token_ides, self.weights):
            x0, x1 = t0, t1
            y0, y1 = Yn[n0], Yn[n1]
            
            edge_trace['x'] += tuple([x0, x1, None])
            edge_trace['y'] += tuple([y0, y1, None])

        node_trace = go.Scatter(
            x=[],
            y=[],
            text=[],
            mode='markers',
            #hoverinfo='text',
            marker=dict(
                showscale=True,
                # colorscale options
                #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
                #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
                #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
                colorscale='YlGnBu',
                reversescale=True,
                color=[],
                size=10,
                colorbar=dict(
                    thickness=15,
                    title='Node Connections',
                    xanchor='left',
                    titleside='right'
                ),
                line=dict(width=2)))

        for i, node in enumerate(markers):
            x, y = i, Yn[node]
            node_trace['x'] += tuple([x])
            node_trace['y'] += tuple([y])

        fig = go.Figure(data=[edge_trace, node_trace])
        plotly.offline.iplot(fig)#, filename='basic-line')



In [173]:
import random
def seq():
    for i in range(100):
        toks = []
        if random.random() < .3:
            toks.append(random.choice("ABC"))
        if random.random() < .7:
            toks.append(random.choice("DEF"))
        if random.random() < .5 and ("D" in toks or "C" in toks):
            toks.append("G")
            
        yield toks

net = RecNet(seq())
net.summary()
net.plot()

tokens= [frozenset({'D', 'G'}), frozenset({'B', 'F'}), frozenset({'B', 'E'}), frozenset({'E'}), frozenset({'D'}), frozenset({'F'}), frozenset({'D'}), frozenset({'C'}), frozenset({'D', 'G'}), frozenset({'D', 'A'}), frozenset({'F'}), frozenset({'F'}), frozenset({'D'}), frozenset({'E'}), frozenset({'E'}), frozenset({'D', 'G'}), frozenset({'C', 'G'}), frozenset({'D'}), frozenset({'E'}), frozenset({'F'}), frozenset({'A'}), frozenset({'E', 'C'}), frozenset({'D', 'G'}), frozenset({'F'}), frozenset({'F'}), frozenset({'D'}), frozenset({'E'}), frozenset({'F', 'A'}), frozenset({'D', 'G'}), frozenset({'E'}), frozenset({'F', 'A'}), frozenset({'B', 'F'}), frozenset({'B'}), frozenset({'E'}), frozenset({'A'}), frozenset({'F', 'C', 'G'}), frozenset({'F'}), frozenset({'E'}), frozenset({'F'}), frozenset({'D', 'G'}), frozenset({'E'}), frozenset({'B', 'F'}), frozenset({'E', 'C', 'G'}), frozenset({'D', 'C'}), frozenset({'E'}), frozenset({'E', 'A'}), frozenset({'D', 'B'}), frozenset({'E'}), frozenset({'D'}),

IndexError: list index out of range

In [171]:
t

NameError: name 't' is not defined

In [108]:
random.choice("ABC")

'A'

In [112]:
len("094-099 {D}")

11

In [121]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.

